In [4]:
!pip install wikipedia

In [105]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mirun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mirun\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\mirun\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\mirun\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
import gensim
from gensim.models import Word2Vec

modelPath = './GoogleNews-vectors-negative300.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(modelPath, binary=True)

In [127]:
import wikipedia
import nltk
import string

nlp = wikipedia.page("Natural language processing")
original_text = nltk.tokenize.sent_tokenize(nlp.content)[:19]
text = [word for word in nltk.word_tokenize(' '.join(original_text)) if word[0] not in string.punctuation]

## 1. Print the number of words in the model's vocabulary

In [128]:
print(f'Model size: {model.vectors.shape[0]}')

Model size: 3000000


## 2. Print all the words in the text that do not appear in the model's vocabulary

In [129]:
outliers = set()

for word in text:
    if word not in model:
        outliers.add(word)

print(outliers)

{'a', 'and', '1966', '1970s', 'ALPAC', 'well-summarized', 'ten-year-long', 'of', 'chatterbots', '–', '1978', '1980s', '1981', 'human-like', 'real-world', 'QUALM', '1975', '1990s', '1960s', 'Rogerian', 'natural-language', '1954', '1950s', '1979', 'hey-day', '1964', '1950', 'e.g.', 'computer-understandable', '1976', '1977', 'SHRDLU', 'to'}


## 3. Which are the two most distant words in the text, and which are the closest? Print the distance too

In [130]:
text = list(set(text) - outliers)
n = len(text)

min_similarity = (1, '', '')
max_similarity = (-1, '', '')

for i in range(n):
    for j in range(i + 1, n):
        similarity = model.similarity(text[i], text[j])
        if similarity > max_similarity[0]:
            max_similarity = (similarity, text[i], text[j])

        if similarity < min_similarity[0]:
            min_similarity = (similarity, text[i], text[j])

print(f'Most distant words: "{min_similarity[1]}", "{min_similarity[2]}" at distance: {min_similarity[0]}')
print(f'Closest words: "{max_similarity[1]}", "{max_similarity[2]}" at distance: {max_similarity[0]}')

Most distant words: "further", "MARGIE" at distance: -0.18977756798267365
Closest words: "three", "five" at distance: 0.9370177388191223


## 4. Using NER (Named Entity Recognition) find the named entities in the text. Print the first 5 most similar words to them both in upper and lowercase

In [131]:
import nltk

ner = []

for sentence in original_text:
    for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sentence))):
        if hasattr(chunk, 'label'):
            ner += ['_'.join(c[0] for c in chunk)]

ner = list(set(ner))
print(ner)

['Alan_Turing', 'Machinery', 'Lehnert', 'Carbonell', 'Cullingford', 'ALPAC', 'Plot_Units', 'Russian', 'John_Searle', 'Symbolic', 'Schank', 'Meehan', 'Politics', 'Turing', 'QUALM', 'Rogerian', 'NLP', 'Georgetown', 'Chinese', 'TaleSpin', 'PAM', 'MARGIE', 'Little', 'SAM', 'Joseph_Weizenbaum', 'Intelligence', 'English', 'ELIZA', 'Wilensky', 'PARRY', 'SHRDLU']


In [111]:
for word in ner:
    uppercase = [elem[0] for elem in model.most_similar(word)[:5]] if word in model else ''
    lowercase = [elem[0] for elem in model.most_similar(word.lower())[:5]] if word.lower() in model else ''

    print(f'Similarity for the word: {word}\n' + \
          f'Most similar in uppercase: {uppercase}\n' + \
          f'Most similar in lowercase: {lowercase}\n')

Similarity for the word: Alan_Turing
Most similar in uppercase: ['Turing', 'Charles_Babbage', 'mathematician_Alan_Turing', 'pioneer_Alan_Turing', 'On_Computable_Numbers']
Most similar in lowercase: 

Similarity for the word: Machinery
Most similar in uppercase: ['Machine_Tool', 'Machinery_SHB_######', 'Machine_Tools', 'Equipment', 'Hydraulics']
Most similar in lowercase: ['machineries', 'equipment', 'Machinery', 'maker_Komatsu_Ltd.', 'Komatsu_####.T']

Similarity for the word: Cullingford
Most similar in uppercase: ['Frappell', 'Skilbeck', 'Griffiths', 'Mayled', 'Forteath']
Most similar in lowercase: 

Similarity for the word: ALPAC
Most similar in uppercase: 
Most similar in lowercase: 

Similarity for the word: Russian
Most similar in uppercase: ['Ukrainian', 'Russia', 'Kazakh', 'Belarusian', 'Belarussian']
Most similar in lowercase: ['russia', 'moscow', 'british', 'russians', 'poland']

Similarity for the word: John_Searle
Most similar in uppercase: 
Most similar in lowercase: 

Sim

## 5. Print the clusters of words that are the most similar in the text (you can use sklearn's Kmeans) based on their vectors in the model

In [172]:
from sklearn.cluster import KMeans
import numpy as np

model['When']
X = np.array([model[word] for word in text])

n_clusters = 6
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(X)

for j in range(n_clusters):
    cluster = []

    for i in range(len(text)):
        if kmeans.labels_[i] == j:
            cluster += [text[i]]

    print(f'Label: {j}\nCluster: {cluster}')

Label: 0
Cluster: ['Symbolic', 'symbolic']
Label: 1
Cluster: ['interactions', 'intelligence', 'applying', 'Turing', 'published', 'Intelligence', 'PARRY', 'science', 'accurately', 'Machinery', 'Challenges', 'human', 'example', 'research', 'generic', 'collection', 'authors', 'premise', 'generation', 'system', 'statistical', 'experiment', 'machine', 'Plot', 'computer', 'MARGIE', 'ELIZA', 'subfield', 'worlds', 'recognition', 'interaction', 'articulated', 'titled', 'speech', 'processing', 'psychotherapist', 'analyze', 'Natural', 'Weizenbaum', 'technology', 'developed', 'PAM', 'Units', 'program', 'History', 'automated', 'criterion', 'emulates', 'conceptual', 'article', 'extract', 'test', 'conducted', 'data', 'computers', 'simulation', 'Computing', 'systems', 'write', 'artificial', 'report', 'categorize', 'Politics', 'Chinese', 'written', 'structured', 'methods', 'SAM', 'programmers']
Label: 2
Cluster: ['When', 'Why', 'However', 'Some', 'My', 'Given', 'Examples', 'During', 'The', 'Using', 'Al